# Actev'21 Visualization

In [1]:
import vipy

In [11]:
videodir = '/path/to/dir/containing/MEVA/videos'
jsonfile = '/path/to/actev_output.json'

In [19]:
# Parse JSON into vipy video objects, with one tracklet per activity
videos = [vipy.video.Scene(filename=k, 
                           tracks=[vipy.object.Track(trackid=str(a['activityID']), attributes={'objectID':o['objectID']}, filterbox=True, category=o['objectType'], keyframes=[int(kf) for (kf,bb) in o['localization'][k].items()], boxes=[vipy.object.Detection(category=o['objectType'], xmin=bb['boundingBox']['x'], ymin=bb['boundingBox']['y'], width=bb['boundingBox']['w'], height=bb['boundingBox']['h']) for bb in list([kb if 'boundingBox' in kb else list(o['localization'][k].values())[-2] for kb in o['localization'][k].values()])]) for a in A for o in a['objects']],
                           activities=[vipy.activity.Activity(category=a['activity'], confidence=a['presenceConf'], attributes={'activityID':a['activityID']}, startframe=int(list(a['localization'][k].keys())[0]), endframe=int(list(a['localization'][k].keys())[1]), tracks=[str(a['activityID'])]) for a in A])
          for (k,A) in vipy.util.groupbyasdict(vipy.util.readjson(jsonfile)['activities'], lambda x: next(iter(x['localization'].keys()))).items()]

print(videos)

[<vipy.video.scene: filename="2018-03-13.20-05-01.20-10-01.school.G328.avi", fps=30.0, tracks=206, activities=206>, <vipy.video.scene: filename="2018-03-14.07-30-04.07-35-04.school.G336.avi", fps=30.0, tracks=12, activities=12>, <vipy.video.scene: filename="2018-03-15.15-55-01.16-00-01.admin.G326.avi", fps=30.0, tracks=26, activities=26>, <vipy.video.scene: filename="VIRAT_S_000206_02_000294_000327.mp4", fps=30.0, tracks=258, activities=258>, <vipy.video.scene: filename="VIRAT_S_040003_02_000197_000552.mp4", fps=30.0, tracks=2978, activities=2978>, <vipy.video.scene: filename="VIRAT_S_040100_03_000496_000559.mp4", fps=30.0, tracks=378, activities=378>]


In [21]:
# Set the absolute path to the video file
d_filename_to_path = {vipy.util.filetail(f):f for f in vipy.util.findvideo(videodir)}
videos = [v.filename(d_filename_to_path[vipy.util.filetail(v.filename())]) for v in videos]

In [23]:
import vipy.data.meva

# (optional) Assign a "shortlabel" to each activity so that it is visualized as "Noun Verb"
videos = [v.activitymap(lambda a: a.shortlabel(vipy.data.meva.d_category_to_shortlabel[a.category()])) for v in videos]                        

In [ ]:
# Generate annotation videos so that minimum dimension is 512px, with only confident activities 
annotations = [(v.clone().mindim(512)               
                 .activityfilter(lambda a: a.confidence() > 0.5)                  
                 .annotate(outfile='%s_annotation.mp4' % v.videoid())) for v in videos]

In [ ]:
# Generate annotation videos so that minimum dimension is 256px, showing only tracks and a timestamp
mutator = vipy.image.mutator_show_noun_only()
annotations = [(v.clone().mindim(256)
                 .activityfilter(lambda a: a.confidence() > 0.5)
                 .annotate(timestamp=True, mutator=mutator, boxalpha=0.0, outfile='%s_trackonly.mp4' % v.videoid())) 
               for v in videos]

In [ ]:
# You may use different mutators to show different properties of the video during annotation
#   - https://visym.github.io/vipy/vipy/image.html#vipy.image.mutator_show_trackindex_verbonly
mutator = vipy.image.mutator_show_trackindex_verbonly()

In [ ]:
import vipy.batch

# Parallel high resolution video annotation with 8 parallel processes
with vipy.globals.parallel(8):
    vipy.batch.Batch(videos).map(lambda v: (v.activityfilter(lambda a: a.confidence()>0.1)
                                             .annotate(outfile='%s_annotation.mp4' % v.videoid())))